In [51]:
import tabula
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

# Файл 1

## Читаем файл

In [150]:
df_pdf = tabula.read_pdf('PP-2022-pril-1.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)

In [151]:
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Код и наименование\rнаправления подготовки/\rс...,Направленность (профиль/специализация) образов...,Форма обучения,Условия\rпоступления,для поступающих на базе\rсреднего общего образ...,для поступающих на базе\rсреднего профессионал...,для поступающих на базе\rвысшего образования,Минимальное\rколичество баллов,Количество\rмест\rдля приема\r(включая особую ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Наименование вступительного\rиспытания\rс указ...,Форма проведения\rвступительного\rиспытания,Наименование вступительного\rиспытания\rс указ...,Форма проведения\rвступительного\rиспытания,Наименование вступительного\rиспытания\rс указ...,Форма проведения\rвступительного\rиспытания,Наименование вступительного\rиспытания\rс указ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,3,4,5,6,7,8,9,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,БАКАЛАВРИАТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.03.02\rПрикладная\rматематика и\rинформатика,Образовательная программа:\rПрикладные цифровы...,очная,договор,1. Математика (профильная),ЕГЭ/ВИ\rдля отдельных категорий\rпоступающих*\...,1. Математика (профильная),ЕГЭ/ВИ\rдля отдельных категорий\rпоступающих*\...,1. Математика (профильная),ЕГЭ/ВИ\rдля отдельных категорий\rпоступающих*\...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,NaN,NaN,2. Профессиональное испытание,NaN,ДВИ\rтестирование,NaN,2. Профессиональное испытание,NaN,ДВИ\rтестирование,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109,NaN,NaN,3. Русский язык,NaN,ЕГЭ/ВИ\rдля отдельных категорий\rпоступающих*\...,NaN,3. Русский язык,NaN,ЕГЭ/ВИ\rдля отдельных категорий\rпоступающих*\...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1111,NaN,договор\r(для иностранных\rграждан и лиц без\r...,NaN,1. Русский язык,NaN,ЕГЭ/ВИ\rтестирование\r(по выбору поступающих),NaN,1. Русский язык,NaN,ЕГЭ/ВИ\rтестирование\r(по выбору поступающих),...,55,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Агрегируем колонки

In [152]:
def func(x):
    try:
        return re.search('\d{2}.\d{2}.\d{2}', x).group(0)
    except:
        return 0

In [209]:
final_df = pd.DataFrame()

Берем только 8 первых элементов из первого столбца, чтобы вытащить код

In [210]:
final_df['Код'] = df[df[0].notna()][0].apply(lambda x: x[:8])

Вытаскиваем код по шаблону

In [211]:
final_df['Код'] = final_df['Код'].apply(lambda x: func(x))

In [212]:
final_df = final_df[final_df['Код'] != 0]

Для направления мы берем все после первого переноса строки `\r` в той же колонке

In [213]:
final_df['Направление'] = df.loc[final_df.index, 0]

In [214]:
final_df['Направление'] = final_df['Направление'].apply(lambda x: ' '.join(x.split('\r')[1:]))

Образовательную программу достаем по тому же принципу

In [215]:
final_df['Образовательная программа'] = df.loc[final_df.index, 1].apply(lambda x: ' '.join(x.split('\r')[1:]))

Подразделение - то что стоит в скобках после образовательной программы

In [216]:
final_df['Подразделение'] = df.loc[final_df.index, 
                                   1].apply(lambda x: re.findall(r'\((.*?)\)',x)).apply(lambda x: x[-1])

In [217]:
final_df['Подразделение'] = final_df['Подразделение'].apply(lambda x: x.replace('\r', ' '))

In [218]:
final_df.head()

,Код,Направление,Образовательная программа,Подразделение
4,01.03.02,Прикладная математика и информатика,Прикладные цифровые технологии (Школа ИТ- мене...,Школа ИТ- менеджмента ЭМИТ
11,09.03.03,Прикладная информатика,1. Прикладная информатика в экономике (ФИТАД Э...,ФИТАД ЭМИТ
24,09.03.03,Прикладная информатика,Прикладная информатика в цифровой экономике (Ф...,ФИТАД ЭМИТ
41,09.03.03,Прикладная информатика,Системы искусственного интеллекта (прикладная ...,ФИТАД ЭМИТ
54,09.03.03,Прикладная информатика,Анализ данных (ФИТАД ЭМИТ),ФИТАД ЭМИТ


Приведем столбец `Образовательная программа` в нужный вид

In [219]:
def func3(x):
    return x['Образовательная программа'].replace(f'({x["Подразделение"]})', '')

In [220]:
final_df['Образовательная программа'] = final_df.apply(lambda x: func3(x), axis=1)

In [221]:
final_df['Образовательная программа'] = final_df['Образовательная программа'].apply(lambda x: x.split('.'))

In [222]:
final_df = final_df.explode('Образовательная программа')

In [223]:
final_df = final_df[~final_df['Образовательная программа'].apply(lambda x: x.isdigit())]

In [224]:
final_df['Образовательная программа'] = final_df['Образовательная программа'].apply(lambda x: x.rstrip(' 1234567890'))

In [225]:
final_df

,Код,Направление,Образовательная программа,Подразделение
4,01.03.02,Прикладная математика и информатика,Прикладные цифровые технологии,Школа ИТ- менеджмента ЭМИТ
11,09.03.03,Прикладная информатика,Прикладная информатика в экономике,ФИТАД ЭМИТ
11,09.03.03,Прикладная информатика,Цифровые технологии,ФИТАД ЭМИТ
11,09.03.03,Прикладная информатика,Прикладная информатика в информационной безоп...,ФИТАД ЭМИТ
24,09.03.03,Прикладная информатика,Прикладная информатика в цифровой экономике,ФИТАД ЭМИТ
...,...,...,...,...
1062,38.05.01,Экономическая безопасность,Обеспечение экономической безопасности бизнеса,ФНБ ИПНБ
1073,38.05.02,Таможенное дело,Организация таможенного контроля,ФТД ИПНБ
1073,38.05.02,Таможенное дело,Таможенные платежи и валютное регулирование,ФТД ИПНБ
1073,38.05.02,Таможенное дело,"Внешнеторговая, транспортная и таможенная лог...",ФТД ИПНБ


# Файл 2

## Читаем файл

In [77]:
df_pdf2 = tabula.read_pdf('PP-2022-pril-2_fil.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df2 = df_pdf2[0]
for df_temp in df_pdf2[1:]:
    df2 = df2.append(df_temp,ignore_index=True)

Got stderr: дек 16, 2021 2:52:42 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [147]:
df2.head(8)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Код и наименование\rнаправления подготовки/\rс...,Форма обучения,Направленность (профиль/специализация)\rобразо...,Условия\rпоступления,для поступающих на базе\rсреднего общего образ...,для поступающих на базе\rсреднего профессионал...,для поступающих на базе\rвысшего образования,Минимальное\rколичество\rбаллов,Количество\rмест\rдля приема\r(включая\rособую...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Наименование\rвступительного\rиспытания,Форма проведения\rвступительного\rиспытания,Наименование\rвступительного\rиспытания,Форма проведения\rвступительного\rиспытания,Наименование\rвступительного\rиспытания,Форма проведения\rвступительного испытания,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,3,4,5,6,7,8,9,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,АЛТАЙСКИЙ ФИЛИАЛ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,БАКАЛАВРИАТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,37.03.01\rПсихология,очная,Образовательная программа:\rПсихология управления,на места в рамках\rконтрольных цифр\rприема\r(...,1. Биология,ЕГЭ\rВИ для отдельных категорий\rпоступающих*\...,1. Биология,ЕГЭ\rВИ для отдельных\rкатегорий поступающих*\...,1. Биология,ЕГЭ\rВИ для отдельных категорий\rпоступающих*\...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,2. Математика (профильная),2. Математика (профильная),2. Математика (профильная),27,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,3. Русский язык,3. Русский язык,3. Русский язык,36,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Агрегируем колонки

Здесь все агрегируется по тому же принципу, что и в первом файле. Кроме подразделения.

In [235]:
final_df2 = pd.DataFrame()

In [236]:
final_df2['Код'] = df2[df2[0].notna()][0].apply(lambda x: x[:8])

In [237]:
final_df2['Код'] = final_df2['Код'].apply(lambda x: func(x))

In [238]:
final_df2 = final_df2[final_df2['Код'] != 0]

In [239]:
final_df2['Направление'] = df2.loc[final_df2.index, 0]

final_df2['Направление'] = final_df2['Направление'].apply(lambda x: ' '.join(x.split('\r')[1:]))

In [240]:
final_df2['Образовательная программа'] = df2.loc[final_df2.index, 2].apply(lambda x: ' '.join(x.split('\r')[1:]))

Чтобы задать подразделение под кажду программу необходимо пробежаться по всем названиям филиалов и вставить для каждой программы соответствующий филиал в зависимости от индекса.

In [241]:
final_df2['Подразделение'] = ''

In [242]:
df_filials = df2[df2[0].notna()][df2[df2[0].notna()][0].str.contains('ФИЛИАЛ')][[0]]

In [243]:
df_filials.head()

,0
3,АЛТАЙСКИЙ ФИЛИАЛ
71,АСТРАХАНСКИЙ ФИЛИАЛ
103,БАЛАКОВСКИЙ ФИЛИАЛ
153,БРЯНСКИЙ ФИЛИАЛ
198,ВЛАДИМИРСКИЙ ФИЛИАЛ


Заполним все филиалы для каждого направления

In [244]:
filial_1 = list(df_filials[0])[0]
for filial in list(df_filials[0])[1:]:
    start_index = (df_filials[df_filials[0] == filial_1]).index[0]
    end_index = (df_filials[df_filials[0] == filial]).index[0]
    
    final_df2.loc[(final_df2.index > start_index)&(final_df2.index < end_index)
                    ,'Подразделение'] = filial_1
    
    filial_1 = filial

final_df2.loc[(final_df2.index > end_index),'Подразделение'] = filial_1

In [245]:
final_df2.head()

,Код,Направление,Образовательная программа,Подразделение
5,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
8,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
11,38.03.01,Экономика,1. Экономика и управление финансами 2. Налоги ...,АЛТАЙСКИЙ ФИЛИАЛ
14,38.03.01,Экономика,1. Экономика и управление финансами 2. Налоги ...,АЛТАЙСКИЙ ФИЛИАЛ
17,38.03.04,Государственное и муниципальное управление,Административное управление,АЛТАЙСКИЙ ФИЛИАЛ


Приведем столбец `Образовательная программа` в нужный вид

In [246]:
final_df2['Образовательная программа'] = final_df2['Образовательная программа'].apply(lambda x: x.split('.'))

In [247]:
final_df2 = final_df2.explode('Образовательная программа')

In [248]:
final_df2 = final_df2[~final_df2['Образовательная программа'].apply(lambda x: x.isdigit())]

In [249]:
final_df2['Образовательная программа'] = final_df2['Образовательная программа'].apply(lambda x: x.rstrip(' 1234567890'))

In [250]:
final_df2

,Код,Направление,Образовательная программа,Подразделение
5,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
8,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
11,38.03.01,Экономика,Экономика и управление финансами,АЛТАЙСКИЙ ФИЛИАЛ
11,38.03.01,Экономика,Налоги и налогообложение,АЛТАЙСКИЙ ФИЛИАЛ
14,38.03.01,Экономика,Экономика и управление финансами,АЛТАЙСКИЙ ФИЛИАЛ
...,...,...,...,...
2873,40.05.01,Правовое обеспечение национальной безопасности,Государственно-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
2879,38.05.01,Экономическая безопасность,Экономико-правовое обеспечение экономической б...,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
2885,40.05.01,Правовое обеспечение национальной безопасности,Гражданско-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
2885,40.05.01,Правовое обеспечение национальной безопасности,Уголовно-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ


# Объединим

In [251]:
df_final = pd.concat([final_df, final_df2])

In [252]:
df_final.shape

(945, 4)

In [253]:
df_final = df_final.drop_duplicates()

In [256]:
df_final = df_final.reset_index(drop='index')

Заменим "ё" на "е" везде

In [259]:
for col in df_final.columns:
    df_final[col] = df_final[col].apply(lambda x: x.replace('ё', 'е'))

In [260]:
df_final.shape

(528, 4)

In [261]:
df_final

,Код,Направление,Образовательная программа,Подразделение
0,01.03.02,Прикладная математика и информатика,Прикладные цифровые технологии,Школа ИТ- менеджмента ЭМИТ
1,09.03.03,Прикладная информатика,Прикладная информатика в экономике,ФИТАД ЭМИТ
2,09.03.03,Прикладная информатика,Цифровые технологии,ФИТАД ЭМИТ
3,09.03.03,Прикладная информатика,Прикладная информатика в информационной безоп...,ФИТАД ЭМИТ
4,09.03.03,Прикладная информатика,Прикладная информатика в цифровой экономике,ФИТАД ЭМИТ
...,...,...,...,...
523,38.05.01,Экономическая безопасность,Финансовый учет и контроль в правоохранительн...,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
524,40.05.01,Правовое обеспечение национальной безопасности,Гражданско-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
525,40.05.01,Правовое обеспечение национальной безопасности,Уголовно-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
526,40.05.01,Правовое обеспечение национальной безопасности,Государственно-правовая специализация,ЮЖНО-РОССИЙСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
